# MLflow results presentation

Present MLflow results using pandas.

If tensorboard is not showing inline, use `!` instead of `%` and view tensorboard in a new browser tab.

In [1]:
import pandas as pd
from uncertainties import ufloat
import mlflow
from mlflow.tracking import MlflowClient
mlflow.set_tracking_uri('file:///home/zeyusun/work/flare-prediction-smarp/mlruns')
client = MlflowClient()

from mlflow_helper import *

%load_ext tensorboard
%load_ext autoreload
%autoreload 2

Display boarderlines of pandas tables

In [2]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

## experiment: beta
* no random seed
* multiple databases

In [3]:
columns = {
    'tags.database_name': 'database',
    'tags.dataset_name': 'dataset',
    'tags.estimator_name': 'estimator',
    'metrics.auc': 'auc',
    'metrics.tss_opt': 'tss',
}
rows = {
    'estimator': {
        'HistGradientBoostingClassifier': 'HGB',
        'RandomForestClassifier': 'RF',
        'SGDClassifier': 'LG',
    }
}
runs_raw = retrieve('experiment', 'beta')
runs = select(runs_raw, columns, rows)
runs

Select the first from 
                          start_time tags.mlflow.runName  \
90  2021-05-22 18:13:26.644000+00:00                beta   
166 2021-05-21 01:50:33.010000+00:00                beta   

                tags.mlflow.source.git.commit  
90   d5d0b545f888847c72dd44d581a77e3e7602b2fe  
166  7149bc5e0355ab3dbc90c029260325a64a1f6044  


,database,dataset,estimator,auc,tss
36,M_Q_6hr,combined,HGB,0.999919,0.998669
37,M_Q_6hr,combined,RF,1.000000,1.000000
38,M_Q_6hr,combined,LG,0.992040,0.921998
39,M_Q_6hr,sharp,HGB,0.996281,0.943164
40,M_Q_6hr,sharp,RF,1.000000,1.000000
41,M_Q_6hr,sharp,LG,0.992198,0.921169
42,M_Q_6hr,smarp,HGB,0.996993,0.950146
43,M_Q_6hr,smarp,RF,1.000000,1.000000
44,M_Q_6hr,smarp,LG,0.992351,0.920737
45,M_Q_12hr_balanced,combined,HGB,1.000000,1.000000


In [4]:
by = ['database', 'dataset', 'estimator']
df_style = style(runs, by=by)
df_style

In [5]:
df = organize(runs, by=by, std=True)
df_latex = typeset(df)
#print(df_latex)
df

database          M_Q_12hr                           M_Q_12hr_balanced  \
dataset           combined        sharp        smarp          combined   
    estimator                                                            
auc HGB        1.000+/-nan  0.994+/-nan  0.996+/-nan       1.000+/-nan   
    LG         0.992+/-nan  0.992+/-nan  0.991+/-nan       0.994+/-nan   
    RF         1.000+/-nan  1.000+/-nan  1.000+/-nan       1.000+/-nan   
tss HGB        1.000+/-nan  0.928+/-nan  0.942+/-nan       1.000+/-nan   
    LG         0.910+/-nan  0.909+/-nan  0.911+/-nan       0.915+/-nan   
    RF         1.000+/-nan  1.000+/-nan  1.000+/-nan       1.000+/-nan   

database                                    M_Q_24hr               \
dataset              sharp        smarp     combined        sharp   
    estimator                                                       
auc HGB        0.997+/-nan  0.998+/-nan  1.000+/-nan  0.999+/-nan   
    LG         0.992+/-nan  0.993+/-nan  0.991+/-nan  0.990+/-nan   
    RF         1.000+/-nan  1.000+/-nan  1.000+/-nan  1.000+/-nan   
tss HGB        0.940+/-nan  0.959+/-nan  1.000+/-nan  0.966+/-nan   
    LG         0.918+/-nan  0.912+/-nan  0.912+/-nan  0.912+/-nan   
    RF         1.000+/-nan  1.000+/-nan  1.000+/-nan  1.000+/-nan   

database                   M_Q_24hr_balanced                            \
dataset              smarp          combined        sharp        smarp   
    estimator                                                            
auc HGB        0.996+/-nan       1.000+/-nan  0.994+/-nan  0.995+/-nan   
    LG         0.991+/-nan       0.991+/-nan  0.991+/-nan  0.991+/-nan   
    RF         1.000+/-nan       1.000+/-nan  1.000+/-nan  1.000+/-nan   
tss HGB        0.940+/-nan       1.000+/-nan  0.926+/-nan  0.938+/-nan   
    LG         0.912+/-nan       0.919+/-nan  0.915+/-nan  0.917+/-nan   
    RF         1.000+/-nan       1.000+/-nan  1.000+/-nan  1.000+/-nan   

database           M_Q_6hr                           M_Q_6hr_balanced  \
dataset           combined        sharp        smarp         combined   
    estimator                                                           
auc HGB        1.000+/-nan  0.996+/-nan  0.997+/-nan      1.000+/-nan   
    LG         0.992+/-nan  0.992+/-nan  0.992+/-nan      0.991+/-nan   
    RF         1.000+/-nan  1.000+/-nan  1.000+/-nan      1.000+/-nan   
tss HGB        0.999+/-nan  0.943+/-nan  0.950+/-nan      1.000+/-nan   
    LG         0.922+/-nan  0.921+/-nan  0.921+/-nan      0.917+/-nan   
    RF         1.000+/-nan  1.000+/-nan  1.000+/-nan      1.000+/-nan   

database                                 
dataset              sharp        smarp  
    estimator                            
auc HGB        0.994+/-nan  0.995+/-nan  
    LG         0.990+/-nan  0.991+/-nan  
    RF         1.000+/-nan  1.000+/-nan  
tss HGB        0.925+/-nan  0.934+/-nan  
    LG         0.899+/-nan  0.913+/-nan  
    RF         1.000+/-nan  1.000+/-nan

## leaderboard1: sklearn_dataset

In [6]:
columns = {
    'tags.database_name': 'database',
    'tags.dataset_name': 'dataset',
    'tags.estimator_name': 'estimator',
    'metrics.auc': 'auc',
    'metrics.tss_opt': 'tss',
}
rows = {
    'estimator': {
        'HistGradientBoostingClassifier': 'HGB',
        'RandomForestClassifier': 'RF',
        'SGDClassifier': 'LG',
    }
}
runs_raw = retrieve('leaderboard1', 'sklearn_dataset')
runs = select(runs_raw, columns, rows)
runs

,database,dataset,estimator,auc,tss
77,M_Q_24hr,fused_sharp,HGB,0.986385,0.892283
78,M_Q_24hr,fused_sharp,RF,0.974761,0.863344
79,M_Q_24hr,fused_sharp,LG,0.983602,0.895498
80,M_Q_24hr,fused_smarp,HGB,0.947810,0.764423
82,M_Q_24hr,fused_smarp,RF,0.938948,0.725962
83,M_Q_24hr,fused_smarp,LG,0.956173,0.810897
84,M_Q_24hr,sharp,HGB,0.991318,0.908360
85,M_Q_24hr,sharp,RF,0.983958,0.890675
86,M_Q_24hr,sharp,LG,0.987159,0.900322
87,M_Q_24hr,smarp,HGB,0.953228,0.801282


In [7]:
df = organize(runs, std=True)
print(df.to_latex())
df

\begin{tabular}{llllll}
\toprule
    & dataset &    fused\_sharp &    fused\_smarp &          sharp &          smarp \\
{} & estimator &                &                &                &                \\
\midrule
auc & HGB &  0.990+/-0.003 &  0.963+/-0.010 &  0.992+/-0.002 &  0.970+/-0.009 \\
    & LG &  0.988+/-0.003 &  0.970+/-0.008 &  0.991+/-0.003 &  0.971+/-0.009 \\
    & RF &  0.983+/-0.006 &  0.957+/-0.011 &  0.988+/-0.004 &  0.958+/-0.012 \\
tss & HGB &  0.907+/-0.019 &  0.812+/-0.034 &  0.919+/-0.016 &  0.839+/-0.029 \\
    & LG &  0.906+/-0.007 &  0.841+/-0.026 &  0.913+/-0.018 &  0.843+/-0.029 \\
    & RF &  0.884+/-0.030 &  0.786+/-0.038 &  0.901+/-0.023 &  0.797+/-0.040 \\
\bottomrule
\end{tabular}



dataset          fused_sharp    fused_smarp          sharp          smarp
    estimator                                                            
auc HGB        0.990+/-0.003  0.963+/-0.010  0.992+/-0.002  0.970+/-0.009
    LG         0.988+/-0.003  0.970+/-0.008  0.991+/-0.003  0.971+/-0.009
    RF         0.983+/-0.006  0.957+/-0.011  0.988+/-0.004  0.958+/-0.012
tss HGB        0.907+/-0.019  0.812+/-0.034  0.919+/-0.016  0.839+/-0.029
    LG         0.906+/-0.007  0.841+/-0.026  0.913+/-0.018  0.843+/-0.029
    RF         0.884+/-0.030  0.786+/-0.038  0.901+/-0.023  0.797+/-0.040

In [8]:
by = ['database', 'dataset', 'estimator']
style(runs, by=by)

## leaderboard1: arnet_dataset

In [9]:
columns = {
    'tags.dataset_name': 'dataset',
    'tags.estimator_name': 'estimator',
    'metrics.test/auc': 'AUC',
    'metrics.test/tss': 'TSS',
}
rows = {}
runs_raw = retrieve('leaderboard1', 'arnet_dataset')
runs = select(runs_raw, columns, rows)
runs

,dataset,estimator,AUC,TSS
0,fused_smarp,CNN,0.942973,0.753205
1,fused_smarp,CNN,0.961599,0.816193
2,fused_smarp,CNN,0.945356,0.758294
3,fused_smarp,CNN,0.949142,0.773788
4,fused_smarp,CNN,0.929361,0.684978
...,...,...,...,...
75,sharp,MLP,0.976567,0.855305
76,sharp,MLP,0.976531,0.849802
81,sharp,MLP,0.974626,0.815385
91,sharp,MLP,0.986490,0.888298


In [10]:
style(runs)

In [11]:
df = organize(runs, std=True)
print(df.to_latex(multicolumn_format='c'))
df

\begin{tabular}{llllll}
\toprule
    & dataset &    fused\_sharp &    fused\_smarp &          sharp &          smarp \\
{} & estimator &                &                &                &                \\
\midrule
AUC & C3D &  0.959+/-0.020 &  0.941+/-0.013 &  0.903+/-0.051 &  0.928+/-0.031 \\
    & CNN &  0.961+/-0.024 &  0.946+/-0.012 &  0.954+/-0.017 &  0.915+/-0.035 \\
    & LSTM &  0.991+/-0.004 &  0.970+/-0.008 &  0.989+/-0.005 &  0.967+/-0.010 \\
    & MLP &  0.989+/-0.004 &  0.969+/-0.007 &  0.977+/-0.006 &  0.962+/-0.006 \\
TSS & C3D &  0.810+/-0.058 &  0.745+/-0.046 &  0.702+/-0.111 &  0.737+/-0.082 \\
    & CNN &  0.801+/-0.062 &  0.757+/-0.047 &  0.789+/-0.058 &  0.701+/-0.076 \\
    & LSTM &  0.912+/-0.024 &  0.837+/-0.028 &  0.903+/-0.031 &  0.831+/-0.033 \\
    & MLP &  0.906+/-0.015 &  0.833+/-0.026 &  0.855+/-0.027 &  0.807+/-0.028 \\
\bottomrule
\end{tabular}



dataset          fused_sharp    fused_smarp          sharp          smarp
    estimator                                                            
AUC C3D        0.959+/-0.020  0.941+/-0.013  0.903+/-0.051  0.928+/-0.031
    CNN        0.961+/-0.024  0.946+/-0.012  0.954+/-0.017  0.915+/-0.035
    LSTM       0.991+/-0.004  0.970+/-0.008  0.989+/-0.005  0.967+/-0.010
    MLP        0.989+/-0.004  0.969+/-0.007  0.977+/-0.006  0.962+/-0.006
TSS C3D        0.810+/-0.058  0.745+/-0.046  0.702+/-0.111  0.737+/-0.082
    CNN        0.801+/-0.062  0.757+/-0.047  0.789+/-0.058  0.701+/-0.076
    LSTM       0.912+/-0.024  0.837+/-0.028  0.903+/-0.031  0.831+/-0.033
    MLP        0.906+/-0.015  0.833+/-0.026  0.855+/-0.027  0.807+/-0.028

## arnet: fusesize_new_more_dataset

In [12]:
columns = {
    'tags.dataset_name': 'dataset',
    'tags.estimator_name': 'estimator',
    'metrics.test/auc': 'AUC',
    'metrics.test/tss': 'TSS',
}
rows = {}
runs_raw = retrieve('arnet', 'fusesize_new_more_dataset')
runs = select(runs_raw, columns, rows)
runs

,dataset,estimator,AUC,TSS
87,fused_smarp,FusionC3D,0.935290,0.737179
88,fused_smarp,FusionC3D,0.960964,0.823851
89,fused_smarp,FusionC3D,0.963064,0.798578
90,fused_smarp,FusionC3D,0.951671,0.791741
91,fused_smarp,FusionC3D,0.931807,0.756726
...,...,...,...,...
162,sharp,CNN,0.938087,0.716887
163,sharp,CNN,0.886316,0.654784
164,sharp,CNN,0.932091,0.705107
165,sharp,CNN,0.863181,0.664815


In [13]:
df = organize(runs)
df_style = style(runs)
df_style

In [14]:
dirs = tensorboard(runs_raw)
%tensorboard --logdir_spec {dirs}

Reusing TensorBoard on port 6007 (pid 15467), started 5 days, 1:54:44 ago. (Use '!kill 15467' to kill it.)

In [15]:
df = organize(runs, std=True)
print(df.to_latex(multicolumn_format='c'))
df

\begin{tabular}{llllll}
\toprule
    & dataset &    fused\_sharp &    fused\_smarp &          sharp &          smarp \\
{} & estimator &                &                &                &                \\
\midrule
AUC & C3D &  0.959+/-0.030 &  0.948+/-0.023 &  0.916+/-0.043 &  0.947+/-0.011 \\
    & CNN &  0.961+/-0.027 &  0.961+/-0.006 &  0.919+/-0.045 &  0.940+/-0.015 \\
    & FusionC3D &  0.961+/-0.020 &  0.949+/-0.014 &  0.912+/-0.050 &  0.931+/-0.032 \\
    & FusionCNN &  0.961+/-0.027 &  0.960+/-0.010 &  0.934+/-0.023 &  0.940+/-0.007 \\
TSS & C3D &  0.810+/-0.078 &  0.782+/-0.068 &  0.748+/-0.081 &  0.763+/-0.016 \\
    & CNN &  0.802+/-0.076 &  0.795+/-0.028 &  0.713+/-0.066 &  0.747+/-0.057 \\
    & FusionC3D &  0.806+/-0.047 &  0.782+/-0.035 &  0.721+/-0.115 &  0.745+/-0.072 \\
    & FusionCNN &  0.805+/-0.087 &  0.804+/-0.034 &  0.738+/-0.066 &  0.743+/-0.028 \\
\bottomrule
\end{tabular}



dataset          fused_sharp    fused_smarp          sharp          smarp
    estimator                                                            
AUC C3D        0.959+/-0.030  0.948+/-0.023  0.916+/-0.043  0.947+/-0.011
    CNN        0.961+/-0.027  0.961+/-0.006  0.919+/-0.045  0.940+/-0.015
    FusionC3D  0.961+/-0.020  0.949+/-0.014  0.912+/-0.050  0.931+/-0.032
    FusionCNN  0.961+/-0.027  0.960+/-0.010  0.934+/-0.023  0.940+/-0.007
TSS C3D        0.810+/-0.078  0.782+/-0.068  0.748+/-0.081  0.763+/-0.016
    CNN        0.802+/-0.076  0.795+/-0.028  0.713+/-0.066  0.747+/-0.057
    FusionC3D  0.806+/-0.047  0.782+/-0.035  0.721+/-0.115  0.745+/-0.072
    FusionCNN  0.805+/-0.087  0.804+/-0.034  0.738+/-0.066  0.743+/-0.028

In [16]:
style(runs)

## arnet: fusesize_QS

In [17]:
columns = get_columns('arnet')
rows = {}
runs_raw = retrieve('arnet', 'fusesize_QS')
runs = select(runs_raw, columns, rows)
runs

,dataset,estimator,AUC,TSS
6,fused_smarp,FusionC3D,0.870811,0.655346
7,fused_smarp,FusionC3D,0.893403,0.650000
8,fused_smarp,FusionC3D,0.903260,0.651032
9,fused_smarp,FusionC3D,0.917868,0.695096
10,fused_smarp,FusionC3D,0.856200,0.585851
...,...,...,...,...
81,sharp,CNN,0.855643,0.565943
82,sharp,CNN,0.906057,0.701195
83,sharp,CNN,0.861959,0.589474
84,sharp,CNN,0.631436,0.234286


In [18]:
df = organize(runs, std=True)
print(df.to_latex(multicolumn_format='c'))
df

\begin{tabular}{llllll}
\toprule
    & dataset &    fused\_sharp &    fused\_smarp &          sharp &          smarp \\
{} & estimator &                &                &                &                \\
\midrule
AUC & C3D &  0.913+/-0.034 &  0.905+/-0.019 &  0.814+/-0.121 &  0.871+/-0.026 \\
    & CNN &  0.902+/-0.044 &  0.889+/-0.015 &  0.832+/-0.115 &  0.876+/-0.009 \\
    & FusionC3D &  0.903+/-0.036 &  0.888+/-0.025 &  0.853+/-0.042 &  0.851+/-0.057 \\
    & FusionCNN &  0.905+/-0.052 &  0.883+/-0.013 &  0.850+/-0.073 &  0.853+/-0.021 \\
TSS & C3D &  0.697+/-0.042 &  0.670+/-0.026 &  0.521+/-0.160 &  0.597+/-0.056 \\
    & CNN &  0.656+/-0.071 &  0.641+/-0.049 &  0.546+/-0.182 &  0.604+/-0.036 \\
    & FusionC3D &  0.671+/-0.072 &  0.647+/-0.039 &  0.604+/-0.022 &  0.569+/-0.092 \\
    & FusionCNN &  0.685+/-0.102 &  0.632+/-0.015 &  0.560+/-0.155 &  0.564+/-0.043 \\
\bottomrule
\end{tabular}



dataset          fused_sharp    fused_smarp          sharp          smarp
    estimator                                                            
AUC C3D        0.913+/-0.034  0.905+/-0.019  0.814+/-0.121  0.871+/-0.026
    CNN        0.902+/-0.044  0.889+/-0.015  0.832+/-0.115  0.876+/-0.009
    FusionC3D  0.903+/-0.036  0.888+/-0.025  0.853+/-0.042  0.851+/-0.057
    FusionCNN  0.905+/-0.052  0.883+/-0.013  0.850+/-0.073  0.853+/-0.021
TSS C3D        0.697+/-0.042  0.670+/-0.026  0.521+/-0.160  0.597+/-0.056
    CNN        0.656+/-0.071  0.641+/-0.049  0.546+/-0.182  0.604+/-0.036
    FusionC3D  0.671+/-0.072  0.647+/-0.039  0.604+/-0.022  0.569+/-0.092
    FusionCNN  0.685+/-0.102  0.632+/-0.015  0.560+/-0.155  0.564+/-0.043

In [19]:
style(runs)

## leaderboard2: sklearn

In [20]:
columns = get_columns('sklearn')
rows = {
    'estimator': {
        'HistGradientBoostingClassifier': 'HGB',
        'RandomForestClassifier': 'RF',
        'SGDClassifier': 'LG',
    }
}
runs_raw = retrieve('leaderboard2', 'sklearn')
runs = select(runs_raw, columns, rows)
runs

Select the first from 
                          start_time tags.mlflow.runName  \
281 2021-06-04 15:43:34.477000+00:00             sklearn   
465 2021-06-04 00:56:40.893000+00:00             sklearn   

                tags.mlflow.source.git.commit  
281  084b4a84b04953ad4e004885cd4fed94986f5979  
465  9ceec256686c5d7c0580736a3ae6a21f7f45d35c  


,dataset,estimator,AUC,TSS
161,fused_sharp,HGB,0.947099,0.779633
162,fused_sharp,RF,0.928796,0.741235
163,fused_sharp,LG,0.958088,0.772955
164,fused_sharp,HGB,0.958913,0.804781
165,fused_sharp,RF,0.949705,0.776892
...,...,...,...,...
276,smarp,RF,0.957159,0.787253
277,smarp,LG,0.976000,0.839318
278,smarp,HGB,0.973456,0.826233
279,smarp,RF,0.961006,0.799327


In [21]:
df = organize(runs, std=True)
print(df.to_latex(multicolumn_format='c'))
df

\begin{tabular}{llllll}
\toprule
    & dataset &    fused\_sharp &    fused\_smarp &          sharp &          smarp \\
{} & estimator &                &                &                &                \\
\midrule
AUC & HGB &  0.970+/-0.023 &  0.946+/-0.025 &  0.972+/-0.023 &  0.951+/-0.023 \\
    & LG &  0.972+/-0.023 &  0.943+/-0.023 &  0.973+/-0.023 &  0.950+/-0.024 \\
    & RF &  0.960+/-0.028 &  0.932+/-0.031 &  0.964+/-0.027 &  0.933+/-0.031 \\
TSS & HGB &  0.852+/-0.079 &  0.764+/-0.071 &  0.854+/-0.077 &  0.781+/-0.070 \\
    & LG &  0.848+/-0.073 &  0.764+/-0.061 &  0.856+/-0.074 &  0.782+/-0.073 \\
    & RF &  0.828+/-0.077 &  0.725+/-0.087 &  0.838+/-0.074 &  0.732+/-0.084 \\
\bottomrule
\end{tabular}



dataset          fused_sharp    fused_smarp          sharp          smarp
    estimator                                                            
AUC HGB        0.970+/-0.023  0.946+/-0.025  0.972+/-0.023  0.951+/-0.023
    LG         0.972+/-0.023  0.943+/-0.023  0.973+/-0.023  0.950+/-0.024
    RF         0.960+/-0.028  0.932+/-0.031  0.964+/-0.027  0.933+/-0.031
TSS HGB        0.852+/-0.079  0.764+/-0.071  0.854+/-0.077  0.781+/-0.070
    LG         0.848+/-0.073  0.764+/-0.061  0.856+/-0.074  0.782+/-0.073
    RF         0.828+/-0.077  0.725+/-0.087  0.838+/-0.074  0.732+/-0.084

In [22]:
style(runs)

## leaderboad2: arnet

In [23]:
columns = get_columns('arnet')
rows = {}
runs_raw = retrieve('leaderboard2', 'arnet')
runs = select(runs_raw, columns, rows)
runs

Select the first from 
                          start_time tags.mlflow.runName  \
160 2021-06-06 18:45:11.099000+00:00               arnet   
467 2021-06-04 00:56:12.203000+00:00               arnet   

                tags.mlflow.source.git.commit  
160  9ceec256686c5d7c0580736a3ae6a21f7f45d35c  
467  9ceec256686c5d7c0580736a3ae6a21f7f45d35c  


,dataset,estimator,AUC,TSS
0,fused_smarp,C3D,0.894203,0.611321
1,fused_smarp,C3D,0.879053,0.654167
2,fused_smarp,C3D,0.915493,0.666979
3,fused_smarp,C3D,0.919003,0.691898
4,fused_smarp,C3D,0.879287,0.611288
...,...,...,...,...
155,sharp,MLP,0.986885,0.912252
156,sharp,MLP,0.982534,0.891182
157,sharp,MLP,0.986856,0.920923
158,sharp,MLP,0.995346,0.938889


In [24]:
style(runs)

In [25]:
df = organize(runs, std=True)
print(df.to_latex(multicolumn_format='c'))
df

\begin{tabular}{llllll}
\toprule
    & dataset &    fused\_sharp &    fused\_smarp &          sharp &          smarp \\
{} & estimator &                &                &                &                \\
\midrule
AUC & C3D &  0.933+/-0.041 &  0.925+/-0.033 &  0.875+/-0.117 &  0.907+/-0.041 \\
    & CNN &  0.931+/-0.041 &  0.930+/-0.036 &  0.882+/-0.069 &  0.901+/-0.046 \\
    & LSTM &  0.971+/-0.027 &  0.947+/-0.025 &  0.966+/-0.027 &  0.943+/-0.025 \\
    & MLP &  0.970+/-0.026 &  0.947+/-0.026 &  0.964+/-0.031 &  0.944+/-0.027 \\
TSS & C3D &  0.741+/-0.103 &  0.713+/-0.078 &  0.648+/-0.180 &  0.678+/-0.088 \\
    & CNN &  0.722+/-0.089 &  0.725+/-0.093 &  0.652+/-0.144 &  0.663+/-0.094 \\
    & LSTM &  0.853+/-0.082 &  0.778+/-0.074 &  0.834+/-0.089 &  0.773+/-0.073 \\
    & MLP &  0.850+/-0.079 &  0.772+/-0.080 &  0.836+/-0.087 &  0.765+/-0.077 \\
\bottomrule
\end{tabular}



dataset          fused_sharp    fused_smarp          sharp          smarp
    estimator                                                            
AUC C3D        0.933+/-0.041  0.925+/-0.033  0.875+/-0.117  0.907+/-0.041
    CNN        0.931+/-0.041  0.930+/-0.036  0.882+/-0.069  0.901+/-0.046
    LSTM       0.971+/-0.027  0.947+/-0.025  0.966+/-0.027  0.943+/-0.025
    MLP        0.970+/-0.026  0.947+/-0.026  0.964+/-0.031  0.944+/-0.027
TSS C3D        0.741+/-0.103  0.713+/-0.078  0.648+/-0.180  0.678+/-0.088
    CNN        0.722+/-0.089  0.725+/-0.093  0.652+/-0.144  0.663+/-0.094
    LSTM       0.853+/-0.082  0.778+/-0.074  0.834+/-0.089  0.773+/-0.073
    MLP        0.850+/-0.079  0.772+/-0.080  0.836+/-0.087  0.765+/-0.077

In [26]:
dirs = tensorboard(runs_raw)
%tensorboard --logdir_spec {dirs}

Reusing TensorBoard on port 6008 (pid 15489), started 5 days, 1:54:34 ago. (Use '!kill 15489' to kill it.)

## CNN: tune_CNN

In [27]:
columns

{'tags.dataset_name': 'dataset',
 'tags.estimator_name': 'estimator',
 'metrics.test/auc': 'AUC',
 'metrics.test/tss': 'TSS'}

In [28]:
columns = {
    'tags.database_name': 'database',
    'tags.dataset_name': 'dataset',
    'tags.estimator_name': 'estimator',
    'params.DATA.SHRINKAGE': 'shrinkage',
    'params.DATA.THRESH': 'thresh',
    'metrics.test/auc': 'AUC',
    'metrics.test/tss': 'TSS',
}
rows = {}
runs_raw = retrieve('CNN', 'tune_CNN')
runs = select(runs_raw, columns, rows)
runs

,database,dataset,estimator,shrinkage,thresh,AUC,TSS
26,M_QS_24hr,fused_sharp,CNN,1/2,150,0.901782,0.661842
27,M_QS_24hr,fused_sharp,CNN,1/2,150,0.868087,0.609524
28,M_QS_24hr,fused_sharp,CNN,1/2,150,0.922934,0.703030
29,M_QS_24hr,fused_sharp,CNN,log,150,0.929250,0.727632
30,M_QS_24hr,fused_sharp,CNN,log,150,0.849244,0.561905
31,M_QS_24hr,fused_sharp,CNN,log,150,0.905430,0.666667
32,M_QS_24hr,fused_sharp,CNN,1/2,50,0.933029,0.705263
33,M_QS_24hr,fused_sharp,CNN,1/2,50,0.862199,0.571429
34,M_QS_24hr,fused_sharp,CNN,1/2,50,0.933362,0.715152
35,M_QS_24hr,fused_sharp,CNN,log,50,0.916192,0.668421


In [29]:
from uncertainties import ufloat
_runs = runs.copy()
_runs['shrinkage'] = pd.Categorical(_runs['shrinkage'], ['None', '1/2', 'log'])
def _organize(runs, by, std):
    runs = (runs
            #.sort_values(by=by, key=lambda s: s if s.name == 'shrinkage' else s)
            .groupby(by)
            .agg(lambda s: ufloat(s.mean(), s.std()))
            .dropna()
            .unstack([-2, -1])
            .T
    )
    return runs
_organize(_runs, by=['database', 'dataset', 'shrinkage', 'thresh'], std=True)

database                  M_QS_24hr                      M_Q_24hr  \
dataset                 fused_sharp        sharp      fused_sharp   
    shrinkage thresh                                                
AUC None      None      0.90+/-0.06  0.83+/-0.08    0.978+/-0.007   
    1/2       150     0.898+/-0.028  0.81+/-0.08    0.973+/-0.006   
              50        0.91+/-0.04  0.84+/-0.07  0.9767+/-0.0018   
    log       150       0.89+/-0.04  0.80+/-0.06    0.974+/-0.004   
              50        0.89+/-0.04  0.86+/-0.08  0.9785+/-0.0026   
TSS None      None      0.67+/-0.11  0.53+/-0.15      0.86+/-0.04   
    1/2       150       0.66+/-0.05  0.49+/-0.14    0.823+/-0.014   
              50        0.66+/-0.08  0.54+/-0.13    0.830+/-0.014   
    log       150       0.65+/-0.08  0.47+/-0.12    0.820+/-0.029   
              50        0.64+/-0.06  0.59+/-0.12    0.843+/-0.022   

database                             
dataset                       sharp  
    shrinkage thresh                 
AUC None      None    0.932+/-0.010  
    1/2       150     0.961+/-0.020  
              50      0.965+/-0.018  
    log       150     0.968+/-0.008  
              50      0.967+/-0.027  
TSS None      None      0.75+/-0.04  
    1/2       150       0.81+/-0.07  
              50      0.827+/-0.035  
    log       150     0.819+/-0.015  
              50      0.835+/-0.022

In [30]:
_runs = runs.copy()
_runs['shrinkage_thresh'] = _runs['shrinkage'] + '_' + _runs['thresh']
style(_runs, by=['database', 'dataset', 'shrinkage_thresh'])

## CNN: Li2020

In [31]:
columns = {
    'tags.database_name': 'database',
    'tags.dataset_name': 'dataset',
    'tags.estimator_name': 'estimator',
    #'params.DATA.SHRINKAGE': 'shrinkage',
    #'params.DATA.THRESH': 'thresh',
    'metrics.test/auc': 'AUC',
    'metrics.test/tss': 'TSS',
}
rows = {}
runs_raw = retrieve('CNN', 'Li2020')
runs = select(runs_raw, columns, rows)
runs

,database,dataset,estimator,AUC,TSS
0,M_QS_24hr,fused_sharp,CNN_Li2020,0.959915,0.776316
2,M_QS_24hr,fused_sharp,CNN_Li2020,0.840317,0.548571
4,M_QS_24hr,fused_sharp,CNN_Li2020,0.921088,0.711111
6,M_QS_24hr,sharp,CNN_Li2020,0.913765,0.725000
8,M_QS_24hr,sharp,CNN_Li2020,0.809105,0.495238
10,M_QS_24hr,sharp,CNN_Li2020,0.933654,0.745455
12,M_Q_24hr,fused_sharp,CNN_Li2020,0.981138,0.874794
14,M_Q_24hr,fused_sharp,CNN_Li2020,0.988676,0.900000
16,M_Q_24hr,fused_sharp,CNN_Li2020,0.982983,0.900000
18,M_Q_24hr,sharp,CNN_Li2020,0.986070,0.894563


In [32]:
style(runs, by=['database', 'dataset', 'estimator'])

In [33]:
df = organize(runs, by=['database', 'dataset', 'estimator'], std=True)
print(df.to_latex(multicolumn_format='c'))
df

\begin{tabular}{llllll}
\toprule
    & database & \multicolumn{2}{c}{M\_QS\_24hr} & \multicolumn{2}{c}{M\_Q\_24hr} \\
    & dataset &    fused\_sharp &          sharp &    fused\_sharp &          sharp \\
{} & estimator &                &                &                &                \\
\midrule
AUC & CNN\_Li2020 &  0.907+/-0.061 &  0.886+/-0.067 &  0.984+/-0.004 &  0.982+/-0.004 \\
TSS & CNN\_Li2020 &  0.679+/-0.117 &  0.655+/-0.139 &  0.892+/-0.015 &  0.878+/-0.024 \\
\bottomrule
\end{tabular}



database            M_QS_24hr                      M_Q_24hr               
dataset           fused_sharp          sharp    fused_sharp          sharp
    estimator                                                             
AUC CNN_Li2020  0.907+/-0.061  0.886+/-0.067  0.984+/-0.004  0.982+/-0.004
TSS CNN_Li2020  0.679+/-0.117  0.655+/-0.139  0.892+/-0.015  0.878+/-0.024

## CNN: Li2020_convbn

In [34]:
columns = {
    'tags.database_name': 'database',
    'tags.dataset_name': 'dataset',
    'tags.estimator_name': 'estimator',
    #'params.DATA.SHRINKAGE': 'shrinkage',
    #'params.DATA.THRESH': 'thresh',
    'metrics.test/auc': 'AUC',
    'metrics.test/tss': 'TSS',
}
rows = {}
runs_raw = retrieve('CNN', 'Li2020_convbn')
runs = select(runs_raw, columns, rows)
runs

,database,dataset,estimator,AUC,TSS
1,M_QS_24hr,fused_sharp,CNN_Li2020,0.943956,0.755263
3,M_QS_24hr,fused_sharp,CNN_Li2020,0.854973,0.600000
5,M_QS_24hr,fused_sharp,CNN_Li2020,0.935939,0.749495
7,M_QS_24hr,sharp,CNN_Li2020,0.941208,0.742105
9,M_QS_24hr,sharp,CNN_Li2020,0.779258,0.481905
11,M_QS_24hr,sharp,CNN_Li2020,0.932113,0.761616
13,M_Q_24hr,fused_sharp,CNN_Li2020,0.967816,0.803954
15,M_Q_24hr,fused_sharp,CNN_Li2020,0.958759,0.812963
17,M_Q_24hr,fused_sharp,CNN_Li2020,0.979786,0.848780
19,M_Q_24hr,sharp,CNN_Li2020,0.985668,0.904448


In [35]:
df = organize(runs, by=['database', 'dataset', 'estimator'], std=True)
print(df.to_latex(multicolumn_format='c'))
df

\begin{tabular}{llllll}
\toprule
    & database & \multicolumn{2}{c}{M\_QS\_24hr} & \multicolumn{2}{c}{M\_Q\_24hr} \\
    & dataset &    fused\_sharp &          sharp &    fused\_sharp &          sharp \\
{} & estimator &                &                &                &                \\
\midrule
AUC & CNN\_Li2020 &  0.912+/-0.049 &  0.884+/-0.091 &  0.969+/-0.011 &  0.970+/-0.024 \\
TSS & CNN\_Li2020 &  0.702+/-0.088 &  0.662+/-0.156 &  0.822+/-0.024 &  0.850+/-0.082 \\
\bottomrule
\end{tabular}



database            M_QS_24hr                      M_Q_24hr               
dataset           fused_sharp          sharp    fused_sharp          sharp
    estimator                                                             
AUC CNN_Li2020  0.912+/-0.049  0.884+/-0.091  0.969+/-0.011  0.970+/-0.024
TSS CNN_Li2020  0.702+/-0.088  0.662+/-0.156  0.822+/-0.024  0.850+/-0.082

## CNN: CNN_comparedwithLi

In [3]:
columns = {
    'tags.database_name': 'database',
    'tags.dataset_name': 'dataset',
    'tags.estimator_name': 'estimator',
    #'params.DATA.SHRINKAGE': 'shrinkage',
    #'params.DATA.THRESH': 'thresh',
    'metrics.test/auc': 'AUC',
    'metrics.test/tss': 'TSS',
}
rows = {}
runs_raw = retrieve('CNN', 'CNN_comparedwithLi')
runs = select(runs_raw, columns, rows)
runs

,database,dataset,estimator,AUC,TSS
0,M_QS_24hr,fused_sharp,CNN,0.921037,0.685526
1,M_QS_24hr,fused_sharp,CNN,0.805317,0.464762
2,M_QS_24hr,fused_sharp,CNN,0.914888,0.672727
3,M_QS_24hr,sharp,CNN,0.850596,0.567105
4,M_QS_24hr,sharp,CNN,0.731597,0.335238
5,M_QS_24hr,sharp,CNN,0.877594,0.658586
6,M_Q_24hr,fused_sharp,CNN,0.972088,0.848435
7,M_Q_24hr,fused_sharp,CNN,0.959374,0.768519
8,M_Q_24hr,fused_sharp,CNN,0.970654,0.821951
9,M_Q_24hr,sharp,CNN,0.938152,0.736409


In [4]:
df = organize(runs, by=['database', 'dataset', 'estimator'], std=True)
print(df.to_latex(multicolumn_format='c'))
df

\begin{tabular}{llllll}
\toprule
    & database & \multicolumn{2}{c}{M\_QS\_24hr} & \multicolumn{2}{c}{M\_Q\_24hr} \\
    & dataset &    fused\_sharp &          sharp &    fused\_sharp &          sharp \\
{} & estimator &                &                &                &                \\
\midrule
AUC & CNN &  0.880+/-0.065 &  0.820+/-0.078 &  0.967+/-0.007 &  0.945+/-0.006 \\
TSS & CNN &  0.608+/-0.124 &  0.520+/-0.167 &  0.813+/-0.041 &  0.773+/-0.036 \\
\bottomrule
\end{tabular}



database           M_QS_24hr                      M_Q_24hr               
dataset          fused_sharp          sharp    fused_sharp          sharp
    estimator                                                            
AUC CNN        0.880+/-0.065  0.820+/-0.078  0.967+/-0.007  0.945+/-0.006
TSS CNN        0.608+/-0.124  0.520+/-0.167  0.813+/-0.041  0.773+/-0.036